In [1]:
import json
import numpy as np
from sklearn.metrics.pairwise import distance_metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import urllib2
import csv

In [2]:
# 'https://www.airbnb.com/rooms/825531?s=vJx50hcn'

headers = {"User-agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"}
search = input('Enter url: ')
if (search.find('?') == -1):
    search_id = search[search.find('rooms/')+6:]
else:
    search_id = search[search.find('rooms/')+6:search.find('?')]
url = 'https://api.airbnb.com/v2/listings/' + search_id + '?client_id=3092nxybyb0otqw18e8nh5nty&_format=v1_legacy_for_p3'
request = urllib2.Request(url, headers=headers)
data = json.loads(urllib2.urlopen(request).read())
input_description = data['listing']['description']

Enter url: 'https://www.airbnb.com/rooms/825531?s=vJx50hcn'
Apartment is located in the residential neighborhood of Roscoe Village. There are restaurants and bars within a 10 minute walk, but you can easily hop on one of the nearby bus lines to reach any other neighborhood in the city. 

Furnished one bedroom apartment available for short term rent.  Free street parking and convenient to public transit. (Damen, Belmont, Diversey and Clybourn buses).

Bedroom features a full-size bed with fresh linens. Additional sleeping areas available on living room futon.

Wifi and basic cable. Fully equipped kitchen with microwave and AC for the Summer months. 

Located in Roscoe Village, a safe and active neighborhood easily accessible to Wrigley Field, Downtown, and popular bars and restaurants. Awesome taco joint open till 3 AM is my favorite go-to after a night out.

Please help yourself to toiletries in the bathroom and anything in the kitchen that you need for a comfortable stay.

I travel a 

In [13]:
#make a dictionary of descriptions: descriptions[listing_id] = description
#make a tf-idf matrix of words by listings
#cosine similarity

#load the data
with open('data/filtered_nyc_listings.csv') as f:
     NYClistings = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]
        
with open('data/filtered_sf_listings.csv') as f:
     SFlistings = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]
    
data = NYClistings + SFlistings

descript_dict = {}
for d in data:
    key = d['id']
    descript_dict[key]=d['description']

id_to_listing = {}
for d in data:
    id_to_listing[d['id']] = d


In [11]:
listing_index_to_id = {index:listing_id for index, listing_id in enumerate([d['id'] for d in data])}

n_feats = 5000
descriptions = [input_description]+ [descript_dict[d] for d in descript_dict]
listing_by_vocab = np.empty((len(descriptions), n_feats))

tfidf_vec = TfidfVectorizer(min_df = 10, max_df = .8, max_features = n_feats, norm='l2', stop_words = 'english')
listing_by_vocab = tfidf_vec.fit_transform(descriptions)

In [15]:
ranked_list = np.argsort(cosine_similarity(listing_by_vocab[0:1], listing_by_vocab))[0][::-1]

[    0 12376 13084 ..., 28340  5581 36113]


In [23]:
top_ten_idx = ranked_list[1:11] #first element is the input listing itself
top_ten_listings = []  #top ten listings and their data
for i in top_ten:
    listing_data = id_to_listing[listing_index_to_id[i]]
    sub_dict = {k: listing_data[k] for k in ('listing_url', 'description', 'price', 'bedrooms', 'accommodates', 
                                       'summary', 'name')}
    top_ten_listings.append(sub_dict)

#get ids of the listings so can look up other information to display
#need to figure out a way to map back from description to id
#descript dict maps the id key to the description value
#is there a faster way to do cosine similarity? can we construct a tf-idf vector from the input description instead
#of adding it to the tf-idf matrix?

{'description': 'This place is amazing! All yours - not shared. Location: East Village on St. Marks! In the middle of it all. Apartment:  top two floors + balcony;  facing a courtyard - very quiet. Bed: Queen with pillow-top - super-comfortable! Note you will be alone in the apartment. The whole two floors and the bathroom and the balcony will be yours and yours only during the dates you book. I only rent out the space when I travel, so please check the calendar before inquiring. Depending on the length of the stay and the requests of the guests, I free up the closets, bathroom cabinets etc.', 'listing_url': 'https://www.airbnb.com/rooms/9531442', 'price': '$120.00', 'bedrooms': '1', 'accommodates': '3', 'summary': 'This place is amazing! All yours - not shared. Location: East Village on St. Marks! In the middle of it all. Apartment:  top two floors + balcony;  facing a courtyard - very quiet. Bed: Queen with pillow-top - super-comfortable!', 'name': 'Duplex Penthouse in East Village!'